In [3]:
import stac_ipyleaflet

# Use ipyleaflet to visually explore MAAP data

Inspired by leafmap.

In [4]:
import ipywidgets
m = stac_ipyleaflet.StacIpyleaflet(zoom=4, layout=ipywidgets.Layout(height="800px"))
m.draw_biomass_map()
m

StacIpyleaflet(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…